In [ ]:
import numpy as np
import random

class AntColony:
    def __init__(self, distances, n_ants=10, n_iterations=100, alpha=1, beta=2, evaporation_rate=0.5, pheromone_init=1.0):
        self.distances = distances  # Matriz de distancias entre ciudades
        self.n_cities = len(distances)
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha  # Influencia de las feromonas
        self.beta = beta    # Influencia de la heurística (1/distancia)
        self.evaporation_rate = evaporation_rate
        self.pheromone = np.full((self.n_cities, self.n_cities), pheromone_init)
        self.best_path = None
        self.best_path_length = float('inf')
    
    def run(self):
        for _ in range(self.n_iterations):
            paths, lengths = self.construct_solutions()
            self.update_pheromones(paths, lengths)
        return self.best_path, self.best_path_length
    
    def construct_solutions(self):
        paths = []
        lengths = []
        for _ in range(self.n_ants):
            path, length = self.construct_solution()
            paths.append(path)
            lengths.append(length)
            if length < self.best_path_length:
                self.best_path_length = length
                self.best_path = path
        return paths, lengths
    
    def construct_solution(self):
        path = [random.randint(0, self.n_cities - 1)]  # Empezamos en una ciudad aleatoria
        while len(path) < self.n_cities:
            current_city = path[-1]
            next_city = self.select_next_city(current_city, path)
            path.append(next_city)
        path.append(path[0])  # Volvemos al inicio
        length = self.calculate_path_length(path)
        return path, length
    
    def select_next_city(self, current_city, visited):
        unvisited = list(set(range(self.n_cities)) - set(visited))
        probabilities = []
        for city in unvisited:
            pheromone = self.pheromone[current_city][city] ** self.alpha
            visibility = (1 / self.distances[current_city][city]) ** self.beta
            probabilities.append(pheromone * visibility)
        probabilities = np.array(probabilities) / sum(probabilities)
        return np.random.choice(unvisited, p=probabilities)
    
    def update_pheromones(self, paths, lengths):
        self.pheromone *= (1 - self.evaporation_rate)  # Evaporación
        for path, length in zip(paths, lengths):
            for i in range(len(path) - 1):
                self.pheromone[path[i]][path[i + 1]] += 1.0 / length  # Depositar feromonas
                self.pheromone[path[i + 1]][path[i]] += 1.0 / length  # Grafo no dirigido
    
    def calculate_path_length(self, path):
        return sum(self.distances[path[i]][path[i + 1]] for i in range(len(path) - 1))

# Ejemplo de uso con una matriz de distancias
distances = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
])

aco = AntColony(distances, n_ants=5, n_iterations=50, alpha=1, beta=2, evaporation_rate=0.3)
best_path, best_path_length = aco.run()
print("Mejor ruta encontrada:", best_path)
print("Longitud de la mejor ruta:", best_path_length)
